In [ ]:
# Install required libraries
!pip install transformers[torch] datasets evaluate kaggle scikit-learn

In [ ]:
import os
from google.colab import files

# Upload the kaggle.json API key
print("Please upload your kaggle.json file")
files.upload()

In [ ]:
# Set up Kaggle directory and permissions
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download and unzip the dataset
!kaggle datasets download -d ilhamfp31/yelp-review-dataset
!unzip -o yelp-review-dataset.zip

In [ ]:
import pandas as pd

# 1. Load the pre-split data
train_df_raw = pd.read_csv('yelp_review_polarity_csv/train.csv', header=None)
test_df_raw = pd.read_csv('yelp_review_polarity_csv/test.csv', header=None)

# 2. Preprocess and format the data
# Column 0 is the label (1 or 2), Column 1 is the text.
train_df_raw.columns = ['label', 'text']
test_df_raw.columns = ['label', 'text']

# Create our dataframes
# We convert labels: 1 -> 0 (Negative), 2 -> 1 (Positive)
train_df = pd.DataFrame({
    'text': train_df_raw['text'],
    'label': train_df_raw['label'].apply(lambda x: 0 if x == 1 else 1)
})

test_df = pd.DataFrame({
    'text': test_df_raw['text'],
    'label': test_df_raw['label'].apply(lambda x: 0 if x == 1 else 1)
})

In [ ]:
# 3. Verify the data
print(f"Total training reviews: {len(train_df)}")
print(f"Total test reviews: {len(test_df)}")

# We will use a smaller sample to avoid Colab time limits.
TRAIN_SAMPLE_SIZE = 50000
TEST_SAMPLE_SIZE = 5000

# Create the sample using 'random_state=42' so the sample is the same every time
train_df = train_df.sample(n=TRAIN_SAMPLE_SIZE, random_state=42)
test_df = test_df.sample(n=TEST_SAMPLE_SIZE, random_state=42)

print(f"\n- USING A SMALLER SAMPLE")
print(f"New training reviews: {len(train_df)}")
print(f"New test reviews: {len(test_df)}")

In [ ]:
import torch
import numpy as np
import evaluate # Hugging Face's evaluation library
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)

# 1. Define Model and Tokenizer
MODEL_NAME = 'google/mobilebert-uncased'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 2. Convert Pandas to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))

In [ ]:
# 3. Tokenize the Data
def tokenize_function(examples):
    # 'truncation=True' cuts reviews longer than the model's max length
    return tokenizer(examples['text'], truncation=True, max_length=512)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# 4. Define Evaluation Metric
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# 5. Define Training Arguments
training_args = TrainingArguments(
    output_dir="yelp_sentiment_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="none",
)

# 6. Create and Run the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

print(" Starting Fine-Tuning --")
trainer.train()
print(" Fine-Tuning Complete-")

In [ ]:
from transformers import pipeline
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm

# Load a larger, general-purpose model for zero-shot classification
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli",
                      device=0) # Use 0 for GPU

# Define labels
candidate_labels = ["positive", "negative"]

# Running on all 5000 test reviews takes a long time, so we ensure we use the sampled set
test_sample = test_df.copy() # reusing the 5000 sample from earlier

predictions_large_model = []
true_labels_large_model = []

print(f" Running Zero-Shot Evaluation on {len(test_sample)} reviews -")

In [ ]:
# Loop through the sample
for index, row in tqdm(test_sample.iterrows(), total=test_sample.shape[0]):
    text = row['text']
    true_label_int = row['label']

    # Get the model's prediction
    result = classifier(
        text,
        candidate_labels,
        hypothesis_template="The sentiment of this review is {}."
    )

    # The result['labels'][0] is the label with the highest score
    predicted_label_str = result['labels'][0]

    # Convert string "positive"/"negative" back to 0/1
    predicted_label_int = 1 if predicted_label_str == "positive" else 0

    predictions_large_model.append(predicted_label_int)
    true_labels_large_model.append(true_label_int)

In [ ]:
large_model_accuracy = accuracy_score(true_labels_large_model, predictions_large_model)

print(f"\n Evaluating Large Zero-Shot Model (BART-Large) --")
print(f"Test Set (Sample) Accuracy: {large_model_accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(true_labels_large_model, predictions_large_model, target_names=['Negative', 'Positive']))

print(" Final Experiment Results --\n")
print(f"Small, Fine-Tuned Model (MobileBERT):")
print(f" Accuracy on {len(test_df)} reviews: {small_model_accuracy:.4f}")
print("\n")
print(f"Large, Zero-Shot Model (BART-Large):")
print(f" Accuracy on {len(test_sample)} reviews: {large_model_accuracy:.4f}")
print("\n")

print("Conclusion")
if small_model_accuracy > large_model_accuracy:
    print("Hypothesis confirmed: The small model fine-tuned on Yelp data")
    print("outperformed the large, general-purpose model.")
else:
    print("Hypothesis not confirmed: The large, general-purpose model")
    print("was more accurate than the small, fine-tuned model.")

In [ ]:
import torch
import numpy as np
import pandas as pd
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from sklearn.metrics import classification_report
from datasets import Dataset
import evaluate, gc

# Clear memory
del model, trainer, classifier
torch.cuda.empty_cache()
gc.collect()

# Reuse the data from earlier steps
# Ensure labels are int and drop NaNs
train_df["label"] = train_df["label"].astype(int)
test_df["label"] = test_df["label"].astype(int)

# Setup Model + Tokenizer
MODEL_NAME = "google/mobilebert-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Convert to HF datasets
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))

# Tokenize (Reduced max_length for LoRA efficiency)
def tokenize_function(batch):
    return tokenizer(batch["text"], truncation=True, max_length=256)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

print("Loading MobileBERT...")
base_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# Correct LoRA modules for MobileBERT
LORA_TARGET_MODULES = [
    "attention.self.query",
    "attention.self.key",
    "attention.self.value",
    "attention.output.dense"
]

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=LORA_TARGET_MODULES,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    modules_to_save=["classifier"] # ensures classifier head is trained
)

peft_model = get_peft_model(base_model, lora_config)
peft_model.print_trainable_parameters()

In [ ]:
# Training Settings (Optimized for Colab T4)
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return metric.compute(predictions=preds, references=labels)

training_args = TrainingArguments(
    output_dir="mobilebert_lora_yelp",
    learning_rate=3e-4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=False, # IMPORTANT: MobileBERT becomes NaN with fp16!
    logging_steps=50,
    warmup_ratio=0.1,
    optim="adamw_torch",
    report_to="none",
    dataloader_pin_memory=True
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

print("Training started...")
trainer.train()
print("Training finished.")

In [ ]:
# Evaluation
results = trainer.evaluate()
print("\nEvaluation:", results)

preds = trainer.predict(tokenized_test).predictions
pred_labels = np.argmax(preds, axis=1)

print("\nClassification Report:")
#print(classification_report(test_df["label"], pred_labels, target_names=["Negative", "Positive"]))